<a href="https://colab.research.google.com/github/AliMousa11/computer-vision/blob/main/EyeDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import numpy as np
import math

In [2]:
img = Image.open('Image 2.jpeg').convert('L')
img_array = np.array(img)

In [3]:
# len(img_array)
len(img_array[1])

1024

In [4]:
def calculateSumImage(image):
  sum_image = np.zeros_like(image, dtype=np.int64)

  for i in range(image.shape[0]):
      for j in range(image.shape[1]):
          if j == 0:
              sum_image[i, j] = image[i, j]
          else:
              sum_image[i, j] = image[i, j] + sum_image[i, j-1]
  return sum_image

sum_image=calculateSumImage(img_array)
sum_image

array([[   204,    408,    612, ..., 212523, 212725, 212927],
       [   204,    408,    612, ..., 210703, 210905, 211107],
       [   204,    408,    612, ..., 208914, 209116, 209318],
       ...,
       [   219,    438,    657, ..., 175501, 175727, 175953],
       [   218,    436,    654, ..., 175270, 175496, 175722],
       [   218,    436,    654, ..., 175338, 175564, 175790]])

In [5]:
def calculateIntegralImage(sum_image):
  integral_image = np.zeros_like(sum_image, dtype=np.int64)

  for i in range(sum_image.shape[0]):
      for j in range(sum_image.shape[1]):
          if i == 0:
              integral_image[i, j] = sum_image[i, j]
          else:
              integral_image[i, j] = sum_image[i, j] + integral_image[i-1, j]
  return integral_image
integral_image=calculateIntegralImage(sum_image)
integral_image

array([[      204,       408,       612, ...,    212523,    212725,
           212927],
       [      408,       816,      1224, ...,    423226,    423630,
           424034],
       [      612,      1224,      1836, ...,    632140,    632746,
           633352],
       ...,
       [   127562,    255145,    382740, ..., 118972566, 119102246,
        119231912],
       [   127780,    255581,    383394, ..., 119147836, 119277742,
        119407634],
       [   127998,    256017,    384048, ..., 119323174, 119453306,
        119583424]])

In [6]:
def calculateLocalSum(integral_image,x_0,y_0,x_1,y_1):
  # print(x_0)
  # print(y_0)
  # print(x_1)
  # print(y_1)

  local_sum=integral_image[x_1,y_1]-integral_image[x_0-1,y_1]-integral_image[x_1,y_0-1]+ integral_image[x_0-1,y_0-1]
  return local_sum


In [7]:
def detectEye(integral_image,kernel_width):
  kernel_height=0.15*kernel_width
  P1 = (-0.5*kernel_width,-0.5*kernel_height)
  P2 = (-0.05*kernel_width,0)
  P3 = (-0.5*kernel_width,0)
  P4 = (-0.05*kernel_width,0.5*kernel_height)
  P5 = (0.05*kernel_width,-0.5*kernel_height)
  P6 = (0.5*kernel_width,0)
  P7 = (0.05*kernel_width,0)
  P8 = (0.5*kernel_width,0.5*kernel_height)
  P9 = (-0.325*kernel_width,0.833*kernel_height)
  P10 = (-0.225*kernel_width,2*kernel_height)
  P11 = (-0.1*kernel_width,0.833*kernel_height)
  P12 = (0.1*kernel_width,2*kernel_height)
  P13 = (0.225*kernel_width,0.833*kernel_height)
  P14 = (0.325*kernel_width,2*kernel_height)
  rectangles = [
    (P1, P2,  1),  # LS1 (White, weight +1)
    (P3, P4, -1),  # LS2 (Black, weight -1)
    (P5, P6,  1),  # LS3 (White, weight +1)
    (P7, P8, -1),  # LS4 (Black, weight -1)
    (P9, P10, -1), # LS5 (Black, weight -1)
    (P11, P12, 1), # LS6 (White, weight +1)
    (P13, P14, -1) # LS7 (Black, weight -1)
  ]
  max_score = -math.inf
  max_position = (0, 0)
  height, width = integral_image.shape
  print(integral_image.shape)
  for i in range(height):
    for j in range(width):

        current_total_score = 0

        for (P_tl_relative, P_br_relative, weight) in rectangles:
            # Top-left corner
            tl_abs_x = int(i + P_tl_relative[0])
            tl_abs_y = int(j + P_tl_relative[1])
            # Bottom-right corner
            br_abs_x = int(i + P_br_relative[0])
            br_abs_y = int(j + P_br_relative[1])

            if tl_abs_x < 0  or tl_abs_y < 0 or br_abs_x >= height or br_abs_y >= width:
                continue
            # calculateLocalSum(integral_image=integral_image,x_0,y_0,x_1,y_1)
            local_sum = calculateLocalSum(integral_image=integral_image, x_0=tl_abs_x,y_0 = tl_abs_y,x_1= br_abs_x,y_1= br_abs_y)
            current_total_score += local_sum * weight

        if current_total_score > max_score:
            max_score = current_total_score
            max_position = (i,j)

  return max_position
max_position=detectEye(integral_image,250)
max_position

(575, 1024)


(125, 18)

In [8]:
def extractDetectedEye(image,max_position,kernel_width):
  kernel_height=0.15*kernel_width
  center_x = max_position[0]
  center_y = max_position[1]
  x_start=center_x-kernel_width/2
  x_end=center_x+kernel_width/2
  y_start=center_y-kernel_height/2
  y_end=center_y+kernel_height/2
  height, width = image.shape
  if x_start < 0: x_start = 0
  if y_start < 0: y_start = 0
  if x_end > width: x_end = width
  if y_end > height: y_end = height
  eyes=image[y_start:y_end, x_start:x_end]
  return eyes
eyes=extractDetectedEye(img_array,max_position,250)
eyes




TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
import matplotlib.pyplot as plt

eyes_extracted = extractDetectedEye(img_array, max_position, 250)

plt.figure()
plt.imshow(eyes_extracted, cmap='gray')
plt.axis('off')
plt.show()